Interpolation overview

1D:
- Mean
- Nearest Neighbour
- Linear Interpolation

2D:
- Mean
- Nearest Neighbor
- Inverse distance


In [137]:
# Imports
from scipy.interpolate import interp1d
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets

In [138]:
# Necessary functions for calcualting the idw
# https://stackoverflow.com/questions/3104781/inverse-distance-weighted-idw-interpolation-with-python

def simple_idw(x, y, z, xi, yi):
    
    dist = distance_matrix(x, y, xi, yi)

    # In IDW, weights are 1 / distance
    weights = 1.0 / dist

    # Make weights sum to one
    weights /= weights.sum(axis=0)

    # Multiply the weights for each interpolated point by all observed Z-values
    zi = np.dot(weights.T, z)
    return zi

def distance_matrix(x0, y0, x1, y1):
    obs = np.vstack((x0, y0)).T
    interp = np.vstack((x1, y1)).T

    # Make a distance matrix between pairwise observations
    # Note: from <http://stackoverflow.com/questions/1871536>
    # (Yay for ufuncs!)
    d0 = np.subtract.outer(obs[:,0], interp[:,0])
    d1 = np.subtract.outer(obs[:,1], interp[:,1])

    return np.hypot(d0, d1)

In [156]:
# Function for interactive 1D plotting
def interactvie_1d_interpolation(x_mid=4,y_mid=6):
    
    x = np.array([2,x_mid,8])
    y = np.array([2,y_mid,3])
    
    f0 = np.mean(y)
    f1 = interp1d(x, y)
    f2 = interp1d(x, y, kind='nearest')

    xnew = np.linspace(2, 8, num=100, endpoint=True)
    plt.figure(figsize=(8,5))
    plt.plot(x, y, 'o', xnew, f1(xnew), '-', xnew, f2(xnew), '--', xnew, np.zeros((len(xnew)))+f0, ':')
    plt.legend(['data', 'linear', 'nearest', 'average'], loc='best')
    plt.ylim(0,8)
    plt.xlim(1.5,8.5)
    plt.show()

In [157]:
# interactive 1D plotting
widgets.interact(interactvie_1d_interpolation, x_mid=(2.1,7.9,0.1), y_mid=(0.2,7.8,0.1));

interactive(children=(FloatSlider(value=4.0, description='x_mid', max=7.9, min=2.1), FloatSlider(value=6.0, de…

In [158]:
# Function for interactive 2D plotting
def interactvie_2d_interpolation(x_mid=5,y_mid=5):
    
    fig, axs = plt.subplots(1, 3, figsize=(20,10))
    
    grid_x, grid_y = np.mgrid[0:10:100j, 0:10:100j]
    
    x = np.array([2,x_mid,8,3])
    y = np.array([2,y_mid,3,8])
    points = np.array((x,y)).swapaxes(0,1)
    values = np.array([1,2,3,4])
    
    grid_z0 = np.zeros((len(grid_x), len(grid_y)))+np.mean(values)
    grid_z1 = griddata(points, values, (grid_x, grid_y), method='nearest')
    grid_z2 = simple_idw(x=x, y=y, z=values, 
                         xi=grid_y.flatten(), yi=grid_x.flatten())
    grid_z2 = grid_z2.reshape(100,100)

    im0 = axs[0].imshow(grid_z0, extent=(0,10,0,10), origin='lower')
    axs[0].scatter(points[:,0], points[:,1], c="black", s=10)
    axs[0].scatter(x_mid, y_mid, c="red", s=15)
    axs[0].set(title="Mean", ylim=(0,10), xlim=(0,10))
    plt.colorbar(im0, ax=axs[0], orientation='horizontal', shrink=1)
    
    im1 = axs[1].imshow(grid_z1.T, extent=(0,10,0,10), origin='lower')
    axs[1].scatter(points[:,0], points[:,1], c="black", s=10)
    axs[1].scatter(x_mid, y_mid, c="red", s=15)
    axs[1].set(title="Nearest Neighbor", ylim=(0,10), xlim=(0,10))
    plt.colorbar(im1, ax=axs[1], orientation='horizontal', shrink=1)
    
    #Invere distance still missing
    im2 = axs[2].imshow(grid_z2, extent=(0,10,0,10), origin='lower')
    axs[2].scatter(points[:,0], points[:,1], c="black", s=10)
    axs[2].scatter(x_mid, y_mid, c="red", s=15)
    axs[2].set(title="Inverse distance", ylim=(0,10), xlim=(0,10))
    plt.colorbar(im2, ax=axs[2], orientation='horizontal', shrink=1)

    
    plt.show()

In [160]:
# interactive 2D plotting
widgets.interact(interactvie_2d_interpolation, x_mid=(1,9,0.1), y_mid=(1,9,0.1));

interactive(children=(FloatSlider(value=5.0, description='x_mid', max=9.0, min=1.0), FloatSlider(value=5.0, de…